In [2]:
# Needed to switch directory in Google drive so as to import MARL env.

from google.colab import drive 
drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
#%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/gym_continuousDoubleAuction/"

!pwd

!pip install -r requirements.txt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
Obtaining gym_continuousDoubleAuction from git+https://github.com/ChuaCheowHuan/gym-continuousDoubleAuction.git@c897137cbcc93ca71cbd51c27e683c3298f6562d#egg=gym_continuousDoubleAuction (from -r requirements.txt (line 24))
  Cloning https://github.com/ChuaCheowHuan/gym-continuousDoubleAuction.git (to revision c897137cbcc93ca71cbd51c27e683c3298f6562d) to ./src/gym-continuousdoubleauction
  Running command git clone -q https://github.com/ChuaCheowHuan/gym-continuousDoubleAuction.git '/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/src/gym-continuousdoubleauction'
  Running command git checkout -q c897137cbcc93ca71cbd51c27e683c3298f6562d
  Using cached https://files.pythonhosted.org/packages/40/23/53ffe29

In [3]:
!pip show tensorflow
!pip show ray

Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wrapt, absl-py, keras-preprocessing, keras-applications, grpcio, tensorflow-estimator, termcolor, scipy, astor, google-pasta, wheel, opt-einsum, gast, protobuf, six, numpy, tensorboard
Required-by: tensorflow-federated, stable-baselines, magenta, fancyimpute
Name: ray
Version: 0.8.2
Summary: A system for parallel and distributed Python that unifies the ML ecosystem.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: funcsigs, py-spy, click, cloudpickle, aiohttp, packaging, numpy, pyyaml, protobuf, redis, google, grpcio, filelock, pytest, six, jsonschema, colorama
Required-by: 

In [0]:
#CDA_env_disc_RLlib

# https://github.com/ray-project/ray/blob/master/python/ray/rllib/examples/multiagent_cartpole.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
"""Simple example of setting up a multi-agent policy mapping.
Control the number of agents and policies via --num-agents and --num-policies.
This works with hundreds of agents and policies, but note that initializing
many TF policies will take some time.
Also, TF evals might slow down with large numbers of policies. To debug TF
execution, set the TF_TIMELINE_DIR environment variable.
"""

# rllib rollout ~/ray_results/PPO/PPO_MMenv-v0_0_2019-08-09_00-06-10t4b1lscr/checkpoint-1 --run PPO --env MMenv-v0 --steps 100

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse
import gym
import random
import numpy as np

import ray
from ray import tune
from ray.rllib.models import Model, ModelCatalog

from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

from ray.tune.registry import register_env
from ray.rllib.utils import try_import_tf


from ray.rllib.policy.policy import Policy
#from ray.rllib.policy.tf_policy import TFPolicy
#from ray.rllib.policy import Policy


#from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG


import sys
if "../" not in sys.path:
    sys.path.append("../")
#from exchg.x.y import z

#from envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv
from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

tf = try_import_tf()

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--num-agents", type=int, default=4)
parser.add_argument("--num-policies", type=int, default=4)
parser.add_argument("--num-iters", type=int, default=10)
parser.add_argument("--simple", action="store_true")

_StoreTrueAction(option_strings=['--simple'], dest='simple', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [0]:
class CustomModel_disc(Model):
    def _lstm(self, Inputs, cell_size):
        s = tf.expand_dims(Inputs, axis=1, name='time_major')  # [time_step, feature] => [time_step, batch, feature]
        lstm_cell = tf.nn.rnn_cell.LSTMCell(cell_size)
        self.init_state = lstm_cell.zero_state(batch_size=1, dtype=tf.float32)
        # time_major means [time_step, batch, feature] while batch major means [batch, time_step, feature]
        outputs, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=s, initial_state=self.init_state, time_major=True)
        lstm_out = tf.reshape(outputs, [-1, cell_size], name='flatten_rnn_outputs')  # joined state representation
        return lstm_out

    def _build_layers_v2(self, input_dict, num_outputs, options):
        hidden = 512
        cell_size = 256
        #S = input_dict["obs"]
        S = tf.layers.flatten(input_dict["obs"])
        with tf.variable_scope(tf.VariableScope(tf.AUTO_REUSE, "shared"),
                               reuse=tf.AUTO_REUSE,
                               auxiliary_name_scope=False):
            last_layer = tf.layers.dense(S, hidden, activation=tf.nn.relu, name="fc1")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc2")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc3")

        last_layer = self._lstm(last_layer, cell_size)

        output = tf.layers.dense(last_layer, num_outputs, activation=tf.nn.softmax, name="mu")

        return output, last_layer

In [0]:
class CustomModel(TFModelV2):
    """Example of a custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        
        super(CustomModel, self).__init__(obs_space, action_space, num_outputs, model_config, name)
        
        self.model = FullyConnectedNetwork(obs_space, action_space, num_outputs, model_config, name)
        #print('obs_space', obs_space)

        self.register_variables(self.model.variables())

    def forward(self, input_dict, state, seq_lens):
        return self.model.forward(input_dict, state, seq_lens)

    def value_function(self):
        return self.model.value_function()

In [0]:
def make_RandomPolicy(_seed):

    # a hand-coded policy that acts at random in the env (doesn't learn)
    class RandomPolicy(Policy):
        """Hand-coded policy that returns random actions."""
        def __init__(self, observation_space, action_space, config):
            self.observation_space = observation_space
            self.action_space = action_space
            self.action_space.seed(_seed)

        def compute_actions(self,
                            obs_batch,
                            state_batches,
                            prev_action_batch=None,
                            prev_reward_batch=None,
                            info_batch=None,
                            episodes=None,
                            **kwargs):
            """Compute actions on a batch of observations."""
            return [self.action_space.sample() for _ in obs_batch], [], {}

        def learn_on_batch(self, samples):
            """No learning."""
            #return {}
            pass

        def get_weights(self):
            pass

        def set_weights(self, weights):
            pass

    return RandomPolicy

In [9]:
#args = parser.parse_args()

# set log_to_driver=False to off render output so as to prevent browser from hanging.
#ray.init(ignore_reinit_error=True)
#ray.init(ignore_reinit_error=True, num_cpus=2)
#ray.init(ignore_reinit_error=True, webui_host='127.0.0.1', num_cpus=2)
ray.init(ignore_reinit_error=True, log_to_driver=False, webui_host='127.0.0.1', num_cpus=2)

2020-03-17 05:04:10,109	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-17 05:04:10,112	INFO resource_spec.py:212 -- Starting Ray with 6.74 GiB memory available for workers and up to 3.37 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-17 05:04:10,631	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2020-03-17_05-04-10_108965_615/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-17_05-04-10_108965_615/sockets/raylet',
 'redis_address': '172.28.0.2:21092',
 'session_dir': '/tmp/ray/session_2020-03-17_05-04-10_108965_615',
 'webui_url': '127.0.0.1:8265'}

In [0]:
num_agents = 4
num_policies = num_agents
num_iters = 3
simple = False #store_true
#num_of_traders = args.num_agents
num_of_traders = num_agents
tape_display_length = 10 #100
tick_size = 1
init_cash = 1000000
max_step = 700 # per episode #700
episode = 5 #9

In [11]:
single_CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step)
obs_space = single_CDA_env.observation_space
act_space = single_CDA_env.action_space
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step))
ModelCatalog.register_custom_model("model_disc", CustomModel_disc)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
# Each policy can have a different configuration (including custom model)
def gen_policy(i):
    config = {"model": {"custom_model": "model_disc"},
              "gamma": 0.99,}
    return (None, obs_space, act_space, config)

In [0]:
"""
def policy_mapper_0(agent_id):
    if agent_id == 0:
        return "policy_0" # PPO
    elif agent_id == 1:
        return "policy_1" # RandomPolicy
    elif agent_id == 2:
        return "policy_2" # RandomPolicy
    else:
        return "policy_3" # RandomPolicy
"""
def policy_mapper(agent_id):
    for i in range(num_agents):
        if agent_id == i:
            return "policy_{}".format(i)

In [0]:
# Setup PPO with an ensemble of `num_policies` different policies

# Dictionary of policies
#policies = {"policy_{}".format(i): gen_policy(i) for i in range(args.num_policies)}
policies = {"policy_{}".format(i): gen_policy(i) for i in range(num_policies)}

In [15]:
# override policy with random policy
"""
policies["policy_{}".format(num_policies-3)] = (make_RandomPolicy(1), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
policies["policy_{}".format(num_policies-2)] = (make_RandomPolicy(2), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
policies["policy_{}".format(num_policies-1)] = (make_RandomPolicy(3), obs_space, act_space, {}) # random policy stored as the last item in policies dictionary
"""
def set_RandomPolicy(policies):
    for i in range(num_agents):
        # random policy stored as the last item in policies dictionary
        policies["policy_{}".format(num_policies-1)] = (make_RandomPolicy(3), obs_space, act_space, {})

    print('policies:', policies)

    return 0
set_RandomPolicy(policies)

policy_ids = list(policies.keys())

policies: {'policy_0': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_2': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_3': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {})}


In [16]:
tune.run(#PPOTrainer,
          "PPO",
          #"PG",
          #queue_trials=False,
          #resources_per_trial={"cpu": 2,
          #                     "gpu": 0},

          #stop={"training_iteration": args.num_iters},
          #stop={"timesteps_total": max_step,
          #      "training_iteration": num_iters},
          stop={"timesteps_total": max_step * episode},

          config={"env": "continuousDoubleAuction-v0",

                  #"log_level": "DEBUG",
                  #"simple_optimizer": args.simple,
                  #"simple_optimizer": True,
                  #"num_sgd_iter": 10,

                  #"gamma": 0.9,

                  # Number of rollout worker actors to create for parallel sampling.
                  # Setting to 0 will force rollouts to be done in the trainer actor.
                  "num_workers": 1, # Colab (only 2 CPUs or 1 GPU) #1
                  "num_envs_per_worker": 2, #2

                  #"timesteps_per_iteration": max_step,

                  # https://github.com/ray-project/ray/issues/4628
                  "sample_batch_size": 32, # number of environment steps sampled from each environment
                  "train_batch_size": 128, # minibatch size must be >= 128, number of environment steps sampled from all available environments

                  "multiagent": {"policies_to_train": ["policy_0"],
                                 "policies": policies,
                                 #"policy_mapping_fn": tune.function(lambda agent_id: random.choice(policy_ids)),
                                 #"policy_mapping_fn": tune.function(policy_mapper),
                                 "policy_mapping_fn": policy_mapper,
                                },
                  },
          )

#ray.shutdown()

Trial name,status,loc
PPO_continuousDoubleAuction-v0_bd9f9e26,RUNNING,


Result for PPO_continuousDoubleAuction-v0_bd9f9e26:
  custom_metrics: {}
  date: 2020-03-17_05-05-30
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 97e8948c687044d18d08130dbcb41749
  experiment_tag: '0'
  hostname: 01ebcde0b5fb
  info:
    grad_time_ms: 20929.984
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 7.970001220703125
        entropy_coeff: 0.0
        kl: 0.010703801177442074
        policy_loss: -0.13240456581115723
        total_loss: 1153154944.0
        vf_explained_var: 0.0
        vf_loss: 1153154944.0
    load_time_ms: 306.026
    num_steps_sampled: 128
    num_steps_trained: 128
    sample_time_ms: 5266.336
    update_time_ms: 16751.961
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_uti

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_bd9f9e26,RUNNING,172.28.0.2:720,nan,43.5768,128,1


Result for PPO_continuousDoubleAuction-v0_bd9f9e26:
  custom_metrics: {}
  date: 2020-03-17_05-05-54
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 97e8948c687044d18d08130dbcb41749
  experiment_tag: '0'
  hostname: 01ebcde0b5fb
  info:
    grad_time_ms: 20098.692
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 8.010251998901367
        entropy_coeff: 0.0
        kl: 0.009549417532980442
        policy_loss: -0.06775331497192383
        total_loss: 63225132.0
        vf_explained_var: 0.0
        vf_loss: 63225132.0
    load_time_ms: 153.556
    num_steps_sampled: 256
    num_steps_trained: 256
    sample_time_ms: 4749.611
    update_time_ms: 8416.306
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_per

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_bd9f9e26,RUNNING,172.28.0.2:720,nan,67.1626,256,2


...

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_bd9f9e26,RUNNING,172.28.0.2:720,0,679.459,3456,27


Result for PPO_continuousDoubleAuction-v0_bd9f9e26:
  custom_metrics: {}
  date: 2020-03-17_05-16-32
  done: true
  episode_len_mean: 701.0
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 0
  episodes_total: 4
  experiment_id: 97e8948c687044d18d08130dbcb41749
  experiment_tag: '0'
  hostname: 01ebcde0b5fb
  info:
    grad_time_ms: 20202.196
    learner:
      policy_0:
        cur_kl_coeff: 0.675000011920929
        cur_lr: 4.999999873689376e-05
        entropy: 8.040399551391602
        entropy_coeff: 0.0
        kl: 0.019970867782831192
        policy_loss: -0.07380717992782593
        total_loss: 7107307.0
        vf_explained_var: 0.0
        vf_loss: 7107307.0
    load_time_ms: 1.369
    num_steps_sampled: 3584
    num_steps_trained: 3584
    sample_time_ms: 4536.624
    update_time_ms: 83.403
  iterations_since_restore: 28
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 91

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_bd9f9e26,TERMINATED,,0,704.651,3584,28


Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_bd9f9e26,TERMINATED,,0,704.651,3584,28


2020-03-17 05:16:32,974	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.
